# A story of weird behavior of Widgetasitc.

## The traceback

We got tracebacks like that:
```
--- Logging error ---
Traceback (most recent call last):
    
...

  File "/home/jhenner/work/widgetastic.patternfly/src/widgetastic_patternfly/__init__.py", line 348, in __repr__
    return '{}({!r})'.format(type(self).__name__, self.locator)
AttributeError: 'SettingsNavDropdown' object has no attribute 'locator'
Call stack:

...
  
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/lib64/python3.7/site-packages/widgetastic/browser.py", line 456, in move_to_element
    self.logger.debug
    ('move_to_element: %r', locator)
Unable to print the message and arguments - possible formatting error.
Use the traceback above to help find the error.
```

# What is wrong?
Well it takes some reading effort but the important is:

```AttributeError: 'SettingsNavDropdown' object has no attribute 'locator'```

 * The `SettingsNavDropdown` s a descendant of `widgetastic_patternfly.NavDropdown`.
 * It has no `__repr__` defined, so the `NavDropdown` is what we should look at.

In [36]:
from widgetastic_patternfly import NavDropdown
nav = NavDropdown()
nav.locator

AttributeError: 'WidgetDescriptor' object has no attribute 'locator'

It trully seem to have no `locator`.

In [103]:
str(NavDropdown())

'NavDropdown()'

Well it works just fine... No error. Or does it really?

In [38]:
NavDropdown().__repr__

<bound method WidgetDescriptor.__repr__ of NavDropdown()>

Hmm nothing specia... **wait what?! ... `WidgetDescriptor`**? I've just created **`NavDropDown`!** ... Or didn't I ?

In [104]:
type(NavDropdown())

widgetastic.widget.base.WidgetDescriptor

I have checked the `NavDropdown` soure. There was **nothing suspiccious** about it.

Until I have seen:

```python
Init signature: Widget(*args, **kwargs)
Source:        
class Widget(object, metaclass=WidgetMetaclass):
    """Base class for all UI objects.

    Does couple of things:

        * Ensures it gets instantiated with a browser or another widget as parent. If you create an
          instance in a class, it then creates a :py:class:`WidgetDescriptor` which is then invoked
          on the instance and instantiates the widget with underlying browser.
        * Implements some basic interface for all widgets.
        
    If you are inheriting from this class, you **MUST ALWAYS** ensure that the inherited class
    has an init that always takes the ``parent`` as the first argument. You can do that on your
    own, setting the parent as ``self.parent`` or you can do something like this:        
    
   .. code-block:: python

        def __init__(self, parent, arg1, arg2, logger=None):
            super(MyClass, self).__init__(parent, logger=logger)
            # or if you have somehow complex inheritance ...
            Widget.__init__(self, parent, logger=logger)

```

# The magic of widgetastic

In [2]:
from widgetastic_patternfly import NavDropdown, View

class MyView(View):
    widget = NavDropdown()

type(MyView.widget)

widgetastic.widget.base.WidgetDescriptor

## After som more time reading the sources...

 * **Perhaps** I need to instantiate the View
 * I need a `Browser` intance to do it.
 * There were some problems to do it, but today I new I can just use `MagicMock`.

In [3]:
from widgetastic.browser import Browser
from unittest.mock import MagicMock

selenium_mock = MagicMock()
view = MyView(Browser(selenium_mock))
type(view.widget)

widgetastic_patternfly.NavDropdown

**Looks promissing!** We sucessfully created a real `NavDropdown`. Let's try it fails:

```python
view.widget

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)

~/work/types_talk/.venv/lib/python3.7/site-packages/widgetastic_patternfly/__init__.py in __repr__(self)
    346 
    347     def __repr__(self):
--> 348         return '{}({!r})'.format(type(self).__name__, self.locator)
    349 
    350 

AttributeError: 'NavDropdown' object has no attribute 'locator'
```

**OMG! OMG! OMG! OMG! It did Fail**. Let's fix it using some mokey-patching.

In [91]:
def __repr__(self):
    return 'The fixed NavDropdown __repr__'

NavDropdown.__broken_repr__ = NavDropdown.__repr__ 
NavDropdown.__repr__ = __repr__

class MyView(View):
    widget = NavDropdown()

view = MyView(Browser(MagicMock()))
view.widget

The fixed NavDropdown __repr__

## What is happening:
 * The `Widget()` doesn't crate a object of class `Widget`, but a `WidgetDescriptor` object.
 * So the real `Widget` instantiation is delayed to the point of accessing the `WidgetDescriptor` object **inside of the `View` object**

I have got the
 * reproducer
 * the fix

# Making a test for `__repr__`
 * There is **lots of `Widget` descendants** defined in the wrapanapi.
 * The wrapanapi is **quite intricated** pile of **Metaclass** magic **Descriptors** and dynamic code smells like `isinstance` and `hassattr`. I should make a test the __str__ and __repr__ of all the classes it exports. But how?


```python
from widgetastic_patternfly import View, Button, Kebab, Text, ...
def test_str(browser):
    class TestView(View):
        any_button = Button()  # will pick up first button...
        button1 = Button('Default Normal')
        button2 = Button(title='Destructive title')
        button3 = Button(title='noText', classes=[Button.PRIMARY])
        kebab = Kebab(...)
        kebab2 = Kebab(id="")
        ...
    view = TestView(browser)
    for widget in view.any_button, view.button1, view.button2, view.button3, ...:
        assert type(str(widget)) is str
```

Not great heh... mainly there is **no guarantee that newly defined class will get tested.**


# Reuse the View for all Widgets?
 * Well I learned something about what happens **behind the seens** of Python types and classes.
 * We can create the `View` in some dynamic way -- We **don't need to reuse** one `View` which may be dangerous.

In [119]:
def make_me_a_class(bases=tuple(), kw={}):
    class OrdinaryType(*bases, **kw):
        some_attribute = "attribute_value"
    # We can just set the attrs with setattr dynamically...
    return OrdinaryType

make_me_a_class((View,))

__main__.make_me_a_class.<locals>.OrdinaryType

We just created a type/class well let's say **in bit more dynamic way**.


In [120]:
DynamicType = type('TheTestView',
                   (View,),
                   dict(some_attribute="attribute value"))
DynamicType

widgetastic.widget.base.TheTestView

In [121]:
import types

TypesType = types.new_class("TypesTestClass",
                            bases=(View,),
                            kwds=None,
                            exec_body=lambda ns: ns.update(dict(some_attribute="attribute_value")))
TypesType

widgetastic.widget.base.TypesTestClass

So there are **three ways to create a new type/class in Python.**

In [122]:
OrdinaryType.__module__, DynamicType.__module__, TypesType.__module__

('__main__', 'widgetastic.widget.base', 'widgetastic.widget.base')

 This is just a small difference of the behavior. that may bite you some day. Otherwise the **results are the same**


# Getting all the Widgets
To do that, I used `inspect`.

In [180]:
import inspect
import widgetastic_patternfly
from widgetastic_patternfly import Widget

widget_descriptors = {name: cls() for name, cls
        in inspect.getmembers(widgetastic_patternfly)
        if inspect.isclass(cls) and issubclass(cls, Widget)  # Some filtering to get the Widget's descendants only. 
}

list(widget_descriptors.items())[:5]

[('AboutModal', AboutModal()),
 ('Accordion', Accordion()),
 ('AggregateStatusCard', AggregateStatusCard()),
 ('AggregateStatusMiniCard', AggregateStatusMiniCard()),
 ('BarChart', BarChart())]

Let's try to stringify them all:

In [181]:
for obj in list(widget_descriptors.values())[:5]:
    print(f"{str(obj):<30} {repr(obj):<30} {type(obj)}")

AboutModal()                   AboutModal()                   <class 'widgetastic.widget.base.WidgetDescriptor'>
Accordion()                    Accordion()                    <class 'widgetastic.widget.base.WidgetDescriptor'>
AggregateStatusCard()          AggregateStatusCard()          <class 'widgetastic.widget.base.WidgetDescriptor'>
AggregateStatusMiniCard()      AggregateStatusMiniCard()      <class 'widgetastic.widget.base.WidgetDescriptor'>
BarChart()                     BarChart()                     <class 'widgetastic.widget.base.WidgetDescriptor'>


* It did work. But we were stringifying the **`Descriptor`s**.
* We need to put the `Descriptor`s to the `View`. When accessing them, "*they become the `Widget`*".

In [177]:
TestView = type('TheTestView', (View,), widget_objects)

In [182]:
the_view = TestView(Browser(MagicMock()))

for obj in the_view.sub_widgets:
    print(obj)

TypeError: __init__() missing 1 required positional argument: 'name'

```python
for obj in the_view.sub_widgets:
    print(obj)

...

~/work/types_talk/.venv/lib/python3.7/site-packages/widgetastic/widget/base.py in wrapped(self, *args, **kwargs)
     65         new_kwargs = {key: resolve_arg(parent, value) for key, value in kwargs.items()}
     66 
---> 67         return method(self, *new_args, **new_kwargs)
     68     return wrapped
     69 

TypeError: __init__() missing 1 required positional argument: 'name'
```

Didn't work too well. Some needs to be initialized with proper args.

In [186]:
import widgetastic_patternfly as wp
DUMMY = "dummy"

init_values = {
    wp.AggregateStatusCard: dict(name=DUMMY),
    wp.AggregateStatusMiniCard: dict(name=DUMMY, locator=DUMMY),
    wp.BarChart: dict(id=DUMMY),
    wp.BootstrapNav: dict(locator=DUMMY),
    wp.NavDropdown: dict()
}

initiable_widget_descriptors = {
    name: cls(**init_values[cls])
    for name, cls in inspect.getmembers(widgetastic_patternfly)
    if inspect.isclass(cls) and issubclass(cls, Widget) and cls in init_values}

TestView = type('TheTestView', (View,), initiable_widget_descriptors)
the_view = TestView(Browser(MagicMock()))

for widget in the_view.sub_widgets:
    assert isinstance(widget, Widget)
    print(str(widget))

BootstrapNav('dummy')
The fixed NavDropdown __repr__


# Inspect and annotations

In [187]:
import inspect
inspect.signature(widgetastic_patternfly.NavDropdown.__init__)

<Signature (self, parent, id=None, logger=None)>

In [29]:
from typing import Dict

def foo(a: str, b:Dict[str, int]={}) -> int: pass
sig = inspect.signature(foo)
sig.parameters, sig.return_annotation

(mappingproxy({'a': <Parameter "a: str">,
               'b': <Parameter "b: Dict[str, int] = {}">}),
 int)

In [30]:
try:
    from typing import get_args
except ImportError:
    def get_args(t):
        return t.__args__


get_args(sig.parameters['b'].annotation)

(str, int)

In [31]:
from typing import Dict, TypeVar, T, Optional

def foo(a: Optional[T]) -> T: pass
inspect.signature(foo).parameters['a'].annotation.__args__

(~T, NoneType)

# TypesFulfill

In [ ]:
class Name():
    pass


class Fixture:
    def __init__(self, name: Name):
        self.name = name
        pass
    
    def __repr__(self):
        return str(self.name)

def tester(f):
    sig = inspect.signature(f)
    bound = sig.bind(**{n: tester(p.annotation) for n, p in sig.parameters.items()})
    print(bound)
    return f(*bound.args, **bound.kwargs)
        
    
def test_it(name: Name, fixture: Fixture, aaa="ddd"):
    assert fixture
    assert isinstance(name, Name)
    return "PASSED", name, fixture, aaa

tester(test_it)

In [ ]:
from IPython.lib.pretty import pprint

class WidgetDescriptor:
    def __new__(self, *args, **kwargs):
        pprint(locals())
        return "Any random object"
    

class MyView:
    the_attribute = WidgetDescriptor("arg", foo="kwarg")
    
MyView.the_attribute, MyView().the_attribute

In [ ]:
from IPython.lib.pretty import pprint

class Metaclass(type): 
    def __new__(cls, name, bases, attrs):
        """
        Crates new type
        """
        print(f"======Metaclass.new is creating a type named {name}")
        pprint(locals())

    def __call__(self, *args, **kwargs):
        """
        Instantiates the class to an object
        """
        pprint(locals())
        print(f"======Metaclass.call is creating an instance of {self}")
        obj = super().__call__(*args, **kwargs)
        assert isinstance(obj, object)
        return obj

    def __init__(self, *args, **kwargs):
        """
        Fills the __dict__
        """
        print(f"=====Metaclass.init lets the {self} modify itself.")
        pprint(locals())
        
        
class WidgetDescriptor:
    """This class handles instantiating and caching of the widgets on view.

    It stores the class and the parameters it should be instantiated with. Once it is accessed from
    the instance of the class where it was defined on, it passes the instance to the widget class
    followed by args and then kwargs.

    It also acts as a counter, so you can then order the widgets by their "creation" stamp.
    """
    def __init__(self, klass, *args, **kwargs):
        self.klass = klass
        self.args = args
        self.kwargs = kwargs
        
        
class Widget(metaclass=Metaclass):
    foo = Widget()
    
Widget.container

In [ ]:
class View(Widget):
    attribute = "content"

In [ ]:
class LazyType:
    def __new__(cls, *args, **kwargs):
        container = Widget
        if (args and isinstance(args[0], container)) \
                or (isinstance(kwargs.get('parent', None), container)):
            return super().__new__(cls)
        else:
            return WidgetDescriptor(cls, *args, **kwargs)

In [ ]:
LazyType(), LazyType(Widget())

In [ ]:
class Widget:
    lazy_attr = LazyType()
    not_so_lazy_attr = LazyType(Widget())
    
Widget.lazy_attr, Widget.not_so_lazy_attr

In [ ]:
from widgetastic_patternfly import View, Widget
from IPython.lib.pretty import pprint
import inspect


class WidgetDescriptor:
    def __init__(self, klass, *args, **kwargs):
        self.klass = klass
        self.args = args
        self.kwargs = kwargs
        
    def __get__(self, owner, owner_class):
        if obj is None:  # class access
            return self
        else:
            new_object = self.klass(obj, *args, **kwargs)
            return new_object

class Meta(type):
    def __new__(cls, name, bases, attrs):
        new_attrs={}
        for key, value in attrs.items():
            if inspect.isclass(value) and issubclass(value, View):
                new_attrs[key] = WidgetDescriptor(value)

        return super(Meta, cls).__new__(cls, name, bases, new_attrs)

class MyWidget(metaclass=Meta):
    def __new__(self, *args, **kwargs):
        if (args and isinstance(args[0], (Widget))) \
                or ('parent' in kwargs and isinstance(kwargs['parent'], (Widget))):
            return super().__new__(cls)
        else:
            return WidgetDescriptor(cls, *args, **kwargs)
        
        
class MyView:
    the_attribute = WidgetDescriptor()
    
MyView.the_attribute
MyView().the_attribute